The total Hamiltonian of a quantum system is given by

$$
\hat{H}_{\text {Total}}= \hat{H}_{\text {SoI}}+\hat{V}_{\text {IoI}}+\hat{V}_{\text {SoI}}+\hat{H}_{\text {world}}+\hat{V}_{\text {world}}
$$

$\hat{H}_{\text{SoI}}$ - the Hamiltonian of the system of interest, e.g., the radical pair.

$\hat{V}_{\text{IoI}}$ - interactions of interest, e.g., singlet-triplet interconversion.

$\hat{V}_{\text {SoI}}$ - interactions within the system of interest that we do not want to describe explicitly, e.g., the radical pair recombination.

$\hat{V}_{\text {world}}$ - interactions with the rest of the world, e.g., fluorescence.

In the open system approach, we only keep $\hat{H}_{\text {SoI}}+\hat{V}_{\text {IoI}}$ in the Hamiltonian and treat the rest as relaxation.

## Lindblad equation

The evolution of an open quantum system with relaxation is described by the Lindblad master equation

$$
    \dfrac{d}{dt}\hat{\rho}=-\dfrac{i}{\hbar}\left[\hat{H},\hat{\rho}\right]+\dfrac{1}{2}\sum_{n} \left(2\hat{L}_n\hat{\rho} \hat{L}_n^{\dagger}-\hat{\rho}\hat{L}_n^{\dagger}\hat{L}_n-\hat{L}_n^{\dagger}\hat{L}_n\hat{\rho}\right),
$$

where $\hat{L}_n$ is a collapse operator corresponding to the relaxation process $n$. The form of $\hat{L}_n$ depends on the nature of the relaxation process.

Common examples:

- $\hat{L}=\sqrt{\Gamma}|1\rangle\langle2|$ - irreversible transitions from $|2\rangle$ to $|1\rangle$ at a rate $\Gamma$ (e.g., due to spontaneous emission). Corresponds to longitudinal relaxation at a rate $\Gamma$:

    $$
    \dot{\rho}_{11}=\Gamma\rho_{22},\ \dot{\rho}_{22}=-\Gamma\rho_{22},
    $$

and transverse relaxation at a rate $\Gamma/2$:

    $$
     \dot{\rho}_{12}=-\dfrac{\Gamma}{2} \rho_{12},\ \dot{\rho}_{21}=-\dfrac{\Gamma}{2} \rho_{21}\\
    $$

- $\hat{L}=\sqrt{\Gamma}\hat{\sigma}_z=\sqrt{\Gamma}\left(|1\rangle\langle1|-|2\rangle\langle2|\right)$ - dephasing between $|1\rangle$ and $|2\rangle$ at a rate $\Gamma$. Corresponds to transverse relaxation at a rate $\Gamma$.


## Longitudinal relaxation

- Relaxation of $\nu_z$ towards its equilibrium value.
- Can lead to ensemble polarization (e.g., spontaneous emission) as well as depolarization (e.g., thermal relaxation).
- In order to maintain the condition $|\vec{\nu}|\le1$ for an arbitrary initial state, any process leading to longitudinal relaxation always leads to transverse relaxation as well.
- Typically characterized in terms of the relaxation rate $\Gamma_1$ and the relaxation time $T_1$.


## Transverse relaxation

- Relaxation of $\nu_x$ and $\nu_y$.
- Can occur without longitudinal relaxation (e.g., dephasing).
- Leads to ensemble depolarization.
- Typically characterized in terms of the relaxation rate $\Gamma_2$ and the relaxation time $T_2$.
- $\Gamma_2\ge\dfrac{\Gamma_1}{2},\ T_2 \le2T_1$.


## Steady state

If it exists, it is unique and does not depend on the initial state.